# Create tools

In [2]:
import sys
sys.path.insert(0, '..')

from dotenv import load_dotenv
from typing import Any, Dict, List, TypedDict
load_dotenv()


True

In [3]:
import requests
import json
import urllib.parse 
from dataclasses import dataclass
from typing import Dict, Any, List, Optional

In [12]:
target_url= 'https://en.wikipedia.org/w/api.php?action=query&format=json&list=search&srsearch=green+farm+land'

In [13]:
encoded_target_url: str = urllib.parse.quote(target_url, safe="")
json_url: str = f"https://r.jina.ai/{encoded_target_url}"

In [14]:
encoded_target_url

'https%3A%2F%2Fen.wikipedia.org%2Fw%2Fapi.php%3Faction%3Dquery%26format%3Djson%26list%3Dsearch%26srsearch%3Dgreen%2Bfarm%2Bland'

In [15]:
headers: Dict[str, str] = {
        "Accept": "application/json"
    }

print(f"Requesting URL: {json_url}")
print(f"With Headers: {headers}")

Requesting URL: https://r.jina.ai/https%3A%2F%2Fen.wikipedia.org%2Fw%2Fapi.php%3Faction%3Dquery%26format%3Djson%26list%3Dsearch%26srsearch%3Dgreen%2Bfarm%2Bland
With Headers: {'Accept': 'application/json'}


In [16]:
response = requests.get(json_url, headers=headers)
response.raise_for_status()

In [17]:
structured_data: Any = response.json()

In [18]:
structured_data

{'code': 200,
 'status': 20000,
 'data': {'title': '',
  'description': '',
  'url': 'https://en.wikipedia.org/w/api.php?action=query&format=json&list=search&srsearch=green+farm+land',
  'content': '{"batchcomplete":"","continue":{"sroffset":10,"continue":"-||"},"query":{"searchinfo":{"totalhits":25858},"search":[{"ns":0,"title":"Farm","pageid":59790,"size":31939,"wordcount":3714,"snippet":"A farm (also called an agricultural holding) is an area of land that is devoted primarily to agricultural processes with the primary objective of producing","timestamp":"2025-10-11T06:04:43Z"},{"ns":0,"title":"Green Gulch Farm Zen Center","pageid":7468979,"size":15018,"wordcount":1647,"snippet":"Green Gulch Farm Zen Center, or S\\u014dryu-ji (\\u84bc\\u9f8d\\u5bfa Green Dragon Temple) is a Soto Zen practice center located near Muir Beach, California, that practices in","timestamp":"2025-10-23T03:05:29Z"},{"ns":0,"title":"Green and Gold","pageid":78631628,"size":11342,"wordcount":951,"snippet":"farme

In [22]:
structured_data.get('data').get('content')

'{"batchcomplete":"","continue":{"sroffset":10,"continue":"-||"},"query":{"searchinfo":{"totalhits":25858},"search":[{"ns":0,"title":"Farm","pageid":59790,"size":31939,"wordcount":3714,"snippet":"A farm (also called an agricultural holding) is an area of land that is devoted primarily to agricultural processes with the primary objective of producing","timestamp":"2025-10-11T06:04:43Z"},{"ns":0,"title":"Green Gulch Farm Zen Center","pageid":7468979,"size":15018,"wordcount":1647,"snippet":"Green Gulch Farm Zen Center, or S\\u014dryu-ji (\\u84bc\\u9f8d\\u5bfa Green Dragon Temple) is a Soto Zen practice center located near Muir Beach, California, that practices in","timestamp":"2025-10-23T03:05:29Z"},{"ns":0,"title":"Green and Gold","pageid":78631628,"size":11342,"wordcount":951,"snippet":"farmer who bets on the Green Bay Packers to help save his farm. This marks the first theatrical film release for Lindwall. Green and Gold was released","timestamp":"2025-10-29T19:47:18Z"},{"ns":0,"title"

In [23]:
json.loads(structured_data.get('data').get('content'))#.model_load()#.json_get('query')

JSONDecodeError: Expecting ',' delimiter: line 1 column 1809 (char 1808)

In [25]:
import requests
from dataclasses import dataclass
from typing import List
from html import unescape

WIKI_API = "https://en.wikipedia.org/w/api.php"
HEADERS = {"User-Agent": "YourApp/0.1 (you@example.com)"}

@dataclass
class SearchQuery:
    title: str
    snippet: str

def get_search_queries(subject: str) -> List[SearchQuery]:
    """Return titles/snippets for a Wikipedia search query."""
    params = {
        "action": "query",
        "format": "json",
        "list": "search",
        "srsearch": subject,
        "srlimit": 10,
    }
    r = requests.get(WIKI_API, params=params, headers=HEADERS, timeout=15)
    r.raise_for_status()
    data = r.json()  # <-- real JSON, no second json.loads needed

    results: List[SearchQuery] = []
    for item in data.get("query", {}).get("search", []):
        snippet_html = item.get("snippet", "")
        # strip the tiny bit of HTML Wikipedia includes in snippets
        snippet = (snippet_html
                   .replace('<span class="searchmatch">', "")
                   .replace("</span>", ""))
        results.append(SearchQuery(title=item.get("title", ""), snippet=unescape(snippet).strip()))
    return results


In [26]:
get_search_queries("green farm land")

[SearchQuery(title='Farm', snippet='A farm (also called an agricultural holding) is an area of land that is devoted primarily to agricultural processes with the primary objective of producing'),
 SearchQuery(title='Green Gulch Farm Zen Center', snippet='Green Gulch Farm Zen Center, or Sōryu-ji (蒼龍寺 Green Dragon Temple) is a Soto Zen practice center located near Muir Beach, California, that practices in'),
 SearchQuery(title='Green and Gold', snippet='farmer who bets on the Green Bay Packers to help save his farm. This marks the first theatrical film release for Lindwall. Green and Gold was released'),
 SearchQuery(title='Agriculture', snippet='six farms in the world consist of fewer than 2 hectares (4.9 acres) and take up only around 12% of all agricultural land. In terms of total land use,'),
 SearchQuery(title='Greens Farms, Connecticut', snippet="Green's Farms is the oldest neighborhood in the town of Westport in Fairfield County, Connecticut, United States. It was first listed as a

In [30]:
@dataclass
class WebPageContent:
    """
    Container for fetched Wikipedia page content.

    Attributes:
        wiki_url:  Canonical page URL.
        title:     Page title.
        content:   Extracted plain-text intro (may be empty on failure).
    """
    wiki_url: str
    title: str
    content: str

def fetch_wiki_page(title: str) -> WebPageContent:
    """
    Fetch a page’s plain-text introduction from Wikipedia.

    Args:
        title: Exact or close page title (e.g., "Capybara").

    Returns:
        WebPageContent with canonical URL, normalized title, and plain-text extract.
        On failure, returns an 'Error' container with a brief message.
    """
    try:
        # Use extracts API to get a clean intro paragraph as plain text.
        params = {
            "action": "query",
            "format": "json",
            "prop": "extracts",
            "explaintext": 1,
            "redirects": 1,
            "titles": title,
        }
        r = requests.get(WIKI_API, params=params, headers=HEADERS, timeout=15)
        r.raise_for_status()
        data = r.json()  # <-- real JSON, no second json.loads needed
        pages = data.get("query", {}).get("pages", {})

        if not pages:
            return WebPageContent(
                wiki_url="N/A",
                title="Error",
                content=f"No page found for: {title}",
            )

        # pages is a dict keyed by pageid; pick the first
        page = next(iter(pages.values()))
        normalized_title = page.get("title", title)
        extract = page.get("extract", "") or ""

        canonical_url = f"https://en.wikipedia.org/wiki/{normalized_title.replace(' ', '_')}"

        return WebPageContent(
            wiki_url=canonical_url,
            title=normalized_title,
            content=extract.strip(),
        )
    except Exception:
        return WebPageContent(
            wiki_url="N/A",
            title="Error",
            content=f"Error fetching page for: {title}",
        )

In [31]:
fetch_wiki_page("Green Gulch Farm Zen Center")

WebPageContent(wiki_url='https://en.wikipedia.org/wiki/Green_Gulch_Farm_Zen_Center', title='Green Gulch Farm Zen Center', content='Green Gulch Farm Zen Center, or Sōryu-ji (蒼龍寺 Green Dragon Temple) is a Soto Zen practice center located near Muir Beach, California, that practices in the lineage of Shunryu Suzuki. In addition to its Zen training program, the center also manages an organic farm and gardens. Founded in 1972 by the San Francisco Zen Center and Zentatsu Richard Baker, the site is located on 115 acres (0.47 km2) in a valley seventeen miles (27 km) north of San Francisco and offers a variety of workshops and classes throughout the year. The land is an inholding of the Golden Gate National Recreation Area and has much wildlife within its borders. In addition to meditation retreats, offerings include classes and workshops on the Japanese tea ceremony and gardening. While Green Gulch Farm has a residential monastery and retreat center, guest house, and conference center, it has a

In [7]:
target_url= 'https://en.wikipedia.org/w/api.php?action=query&format=json&list=search&srsearch=capybara'

In [15]:
encoded_target_url: str = urllib.parse.quote(target_url, safe="")
json_url: str = f"https://r.jina.ai/{encoded_target_url}"

In [16]:
encoded_target_url

'https%3A%2F%2Fen.wikipedia.org%2Fw%2Fapi.php%3Faction%3Dquery%26format%3Djson%26list%3Dsearch%26srsearch%3Dcapybara'

In [18]:
headers: Dict[str, str] = {
        "Accept": "application/json"
    }

print(f"Requesting URL: {json_url}")
print(f"With Headers: {headers}")

Requesting URL: https://r.jina.ai/https%3A%2F%2Fen.wikipedia.org%2Fw%2Fapi.php%3Faction%3Dquery%26format%3Djson%26list%3Dsearch%26srsearch%3Dcapybara
With Headers: {'Accept': 'application/json'}


In [19]:
response = requests.get(json_url, headers=headers)
response.raise_for_status()

In [21]:
structured_data: Any = response.json()

In [25]:
structured_data

{'code': 200,
 'status': 20000,
 'data': {'title': '',
  'description': '',
  'url': 'https://en.wikipedia.org/w/api.php?action=query&format=json&list=search&srsearch=capybara',
  'content': '{"batchcomplete":"","continue":{"sroffset":10,"continue":"-||"},"query":{"searchinfo":{"totalhits":881},"search":[{"ns":0,"title":"Capybara","pageid":6776,"size":36130,"wordcount":3665,"snippet":"The capybara or greater capybara (Hydrochoerus hydrochaeris) is the largest living rodent, native to South America. It is a member of the genus Hydrochoerus","timestamp":"2025-11-01T21:45:11Z"},{"ns":0,"title":"Lesser capybara","pageid":23188846,"size":5505,"wordcount":578,"snippet":"The lesser capybara (Hydrochoerus isthmius) is a large semi-aquatic rodent found in South America that has vast similarities, yet subtle differences, with","timestamp":"2025-10-29T03:59:29Z"},{"ns":0,"title":"Capybara (disambiguation)","pageid":69085306,"size":516,"wordcount":99,"snippet":"Look up capybara\\u00a0or Capybara i

In [ ]:
json.loads(structured_data.get('data').get('content'))#.model_load()#.json_get('query')

{'batchcomplete': '',
 'continue': {'sroffset': 10, 'continue': '-||'},
 'query': {'searchinfo': {'totalhits': 881},
  'search': [{'ns': 0,
    'title': 'Capybara',
    'pageid': 6776,
    'size': 36130,
    'wordcount': 3665,
    'snippet': 'The capybara or greater capybara (Hydrochoerus hydrochaeris) is the largest living rodent, native to South America. It is a member of the genus Hydrochoerus',
    'timestamp': '2025-11-01T21:45:11Z'},
   {'ns': 0,
    'title': 'Lesser capybara',
    'pageid': 23188846,
    'size': 5505,
    'wordcount': 578,
    'snippet': 'The lesser capybara (Hydrochoerus isthmius) is a large semi-aquatic rodent found in South America that has vast similarities, yet subtle differences, with',
    'timestamp': '2025-10-29T03:59:29Z'},
   {'ns': 0,
    'title': 'Capybara (disambiguation)',
    'pageid': 69085306,
    'size': 516,
    'wordcount': 99,
    'snippet': 'Look up capybara\xa0or Capybara in Wiktionary, the free dictionary. The capybara is a giant cavy ro

In [40]:
content=json.loads(structured_data.get('data').get('content'))

In [46]:
search_list= content.get('query').get('search')
for i in search_list:
    print(i.get('title'), i.get('snippet'))

Capybara The capybara or greater capybara (Hydrochoerus hydrochaeris) is the largest living rodent, native to South America. It is a member of the genus Hydrochoerus
Lesser capybara The lesser capybara (Hydrochoerus isthmius) is a large semi-aquatic rodent found in South America that has vast similarities, yet subtle differences, with
Capybara (disambiguation) Look up capybara or Capybara in Wiktionary, the free dictionary. The capybara is a giant cavy rodent native to South America. Capybara may also refer to:
Capybara Games Capybara Games is an independent game studio based in Toronto, Ontario, Canada. The studio was founded in 2003 from a collection of Toronto IGDA members
Capybara (software) Capybara is a web-based test automation software that simulates scenarios for user stories and automates web application testing for behavior-driven software
Caviidae the domestic guinea pig, wild cavies, and the largest living rodent, the capybara. They are found across South America in open a

In [47]:
@dataclass
class SearchQuery:
    title: str
    snippet: str

In [ ]:
def get_search_queries(subject: str) -> List[SearchQuery]:
    print(f"Searching Wikipedia pages related to: {subject}")
    try:
        target_url= f'https://en.wikipedia.org/w/api.php?action=query&format=json&list=search&srsearch={subject}'
        encoded_target_url: str = urllib.parse.quote(target_url, safe="")
        json_url: str = f"https://r.jina.ai/{encoded_target_url}"

        headers: Dict[str, str] = {
                "Accept": "application/json"
            }

        response = requests.get(json_url, headers=headers)
        response.raise_for_status()

        structured_data: Any = response.json()
        content=json.loads(structured_data.get('data').get('content'))
        search_list= content.get('query').get('search')

        results: List[SearchQuery] = []
        for item in search_list:
            results.append(SearchQuery(title=item.get('title'), snippet=item.get('snippet')))
        print(f"Found {len(results)} results.")
        return results
    except Exception as e:
        title: str = "Error"
        snippet: str = f'Error fetching search results for: {subject}'
        return [SearchQuery(title=title, snippet=snippet)]
    

    


In [49]:
get_search_queries("capybara")

[SearchQuery(title='Capybara', snippet='The capybara or greater capybara (Hydrochoerus hydrochaeris) is the largest living rodent, native to South America. It is a member of the genus Hydrochoerus'),
 SearchQuery(title='Lesser capybara', snippet='The lesser capybara (Hydrochoerus isthmius) is a large semi-aquatic rodent found in South America that has vast similarities, yet subtle differences, with'),
 SearchQuery(title='Capybara (disambiguation)', snippet='Look up capybara\xa0or Capybara in Wiktionary, the free dictionary. The capybara is a giant cavy rodent native to South America. Capybara may also refer to:'),
 SearchQuery(title='Capybara Games', snippet='Capybara Games is an independent game studio based in Toronto, Ontario, Canada. The studio was founded in 2003 from a collection of Toronto IGDA members'),
 SearchQuery(title='Caviidae', snippet='the domestic guinea pig, wild cavies, and the largest living rodent, the capybara. They are found across South America in open areas fro

In [50]:
@dataclass
class WebPageCotent:
    """
    Container for fetched web page content.

    Attributes:
        wiki_url: Original Wikipedia URL requested.
        title:    Extracted title from the response (or 'N/A' on failure).
        content:  Extracted main content/body (may be empty on failure).
    """
    wiki_url: str
    title: str
    content: str

In [52]:
title= " Capybara "
title= title.strip()
title=title.replace(' ', '+')

In [53]:
target_url=f"https://en.wikipedia.org/w/index.php?title={title}&action=raw"
encoded_target_url: str = urllib.parse.quote(target_url, safe="")
json_url: str = f"https://r.jina.ai/{encoded_target_url}"

headers: Dict[str, str] = {
                "Accept": "application/json"
            }

response = requests.get(json_url, headers=headers)
response.raise_for_status()

structured_data: Any = response.json()

In [54]:
structured_data

{'code': 200,
 'status': 20000,
 'data': {'title': '',
  'description': '',
  'url': 'https://en.wikipedia.org/w/index.php?title=Capybara&action=raw',
  'content': '{{Short description|Largest species of rodents}}\n{{Other uses}}\n{{Good article}}\n{{pp|small=yes}}\n{{Use dmy dates|date=July 2022}}\n{{Speciesbox\n| status            = LC\n| status_system     = IUCN3.1\n| status_ref        = {{cite iucn |author=Reid, F. |date=2016 |title=\'\'Hydrochoerus hydrochaeris\'\' |volume=2016 |article-number=e.T10300A22190005 |doi=10.2305/IUCN.UK.2016-2.RLTS.T10300A22190005.en |access-date=19 November 2021}}\n| image             = Hydrochoeris hydrochaeris in Brazil in Petrópolis, Rio de Janeiro, Brazil 09.jpg\n| image_caption     = In [[Petrópolis]], Brazil\n| genus             = Hydrochoerus\n| species           = hydrochaeris\n| authority         = ([[Carl Linnaeus|Linnaeus]], [[12th edition of Systema Naturae|1766]])\n| range_map         = Capybara range.svg\n| range_map_caption = Native ran

In [ ]:
content=structured_data.get('data').get('content')
content

'{{Short description|Largest species of rodents}}\n{{Other uses}}\n{{Good article}}\n{{pp|small=yes}}\n{{Use dmy dates|date=July 2022}}\n{{Speciesbox\n| status            = LC\n| status_system     = IUCN3.1\n| status_ref        = {{cite iucn |author=Reid, F. |date=2016 |title=\'\'Hydrochoerus hydrochaeris\'\' |volume=2016 |article-number=e.T10300A22190005 |doi=10.2305/IUCN.UK.2016-2.RLTS.T10300A22190005.en |access-date=19 November 2021}}\n| image             = Hydrochoeris hydrochaeris in Brazil in Petrópolis, Rio de Janeiro, Brazil 09.jpg\n| image_caption     = In [[Petrópolis]], Brazil\n| genus             = Hydrochoerus\n| species           = hydrochaeris\n| authority         = ([[Carl Linnaeus|Linnaeus]], [[12th edition of Systema Naturae|1766]])\n| range_map         = Capybara range.svg\n| range_map_caption = Native range\n| synonyms          = \'\'Sus hydrochaeris\'\' {{small|Linnaeus,\xa01766}}\n}}\n\nThe \'\'\'capybara\'\'\'{{efn | Also referred as \'\'\'capivara\'\'\' (in Braz

In [59]:
def fetch_wiki_pages(title: str) -> WebPageCotent:
    print(f"Fetching Wikipedia page for: {title}")
    title= title.strip()
    title=title.replace(' ', '+')
    try:
        target_url=f"https://en.wikipedia.org/w/index.php?title={title}&action=raw"
        encoded_target_url: str = urllib.parse.quote(target_url, safe="")
        json_url: str = f"https://r.jina.ai/{encoded_target_url}"

        headers: Dict[str, str] = {
                "Accept": "application/json"
            }

        response = requests.get(json_url, headers=headers)
        response.raise_for_status()

        structured_data: Any = response.json()
        content=structured_data.get('data').get('content')

    
        return WebPageCotent(
                wiki_url=target_url,
                title=title,
                content=content
            )
    except Exception as e:
        return WebPageCotent(
            wiki_url="N/A",
            title="Error",
            content=f'Error fetching page for: {title}'
        )

In [5]:
fetch_wiki_pages("green farm land")

Fetching Wikipedia page for: green farm land


WebPageContent(wiki_url='https://en.wikipedia.org/w/index.php?title=green+farm+land&action=raw', title='green+farm+land', content='If you report this error to the Wikimedia System Administrators, please include the details below.\n\n`Request served via cp2037 cp2037, Varnish XID 493861534Upstream caches: cp2037 intError: 404, Not Found at Fri, 07 Nov 2025 09:22:16 GMT`\n\n`Sensitive client information``IP address: 2600:1900:0:2d04::200`')

In [2]:
import sys
sys.path.insert(0, '..')
from tools import fetch_wiki_page, get_search_queries


In [4]:
get_search_queries("green farm land")

[SearchQuery(title='Farm', snippet='A farm (also called an agricultural holding) is an area of land that is devoted primarily to agricultural processes with the primary objective of producing'),
 SearchQuery(title='Green Gulch Farm Zen Center', snippet='Green Gulch Farm Zen Center, or Sōryu-ji (蒼龍寺 Green Dragon Temple) is a Soto Zen practice center located near Muir Beach, California, that practices in'),
 SearchQuery(title='Green and Gold', snippet='farmer who bets on the Green Bay Packers to help save his farm. This marks the first theatrical film release for Lindwall. Green and Gold was released'),
 SearchQuery(title='Greens Farms, Connecticut', snippet="Green's Farms is the oldest neighborhood in the town of Westport in Fairfield County, Connecticut, United States. It was first listed as a census-designated"),
 SearchQuery(title='Hobby farm', snippet='land for horses, or as working farms for secondary income. Hobby farms are agricultural land smaller than a fully-fledged farm. As s

In [3]:
fetch_wiki_page("Capybara (disambiguation)")

WebPageContent(wiki_url='https://en.wikipedia.org/wiki/Capybara_(disambiguation)', title='Capybara (disambiguation)', content='The capybara is a giant cavy rodent native to South America.\nCapybara  may also refer to:\n\nCapybara (software), a web-based test automation software\nCapybara Games, a Canadian independent game studio\n"Capybara" (Sons of Anarchy), an episode of American television drama series Sons of Anarchy\nThe lesser capybara, a large semi-aquatic rodent closely related to the capybara')

# Build Agent

In [6]:
from dataclasses import dataclass

from pydantic import BaseModel

from pydantic_ai import Agent
from pydantic_ai.messages import FunctionToolCallEvent
from pydantic_ai.messages import ModelMessage, UserPromptPart

import tools

In [ ]:
instructions = """
You are an AI assistant specialized in helping users find accurate, relevant information from Wikipedia.

Your job is to:
1. Understand the user’s question.
2. Search Wikipedia for relevant pages.
3. Retrieve the most useful full pages.
4. Synthesize the information into a clear, correct answer.

You have access to two tools:

1. search_wikipedia(subject: str) -> List[SearchQuery]
   - Searches Wikipedia for pages related to a topic.
   - Returns SearchQuery objects containing a page title and snippet.

2. fetch_wikipedia_page(title: str) -> WebPageContent
   - Retrieves the full content of a Wikipedia page.
   - Returns the page title, URL, and content text.

-----------------------------
SEARCH & ANSWERING STRATEGY
-----------------------------

**Step 1 — Understand the question**
- Identify the key topic, subject, or entity (usually 1–3 words).
- If necessary, rewrite the question into a simple search phrase.

**Step 2 — Perform a Wikipedia search**
- Call search_wikipedia using the identified subject.
- Inspect the returned titles and snippets.
- Select the most relevant 1–3 pages.

**Step 3 — Fetch full pages**
- For each selected title, call fetch_wikipedia_page to retrieve detailed content.
- You may use this tool multiple times, but **no more than 5 calls total**.

**Step 4 — Synthesize**
- Read the fetched content.
- Create a concise, accurate answer that directly addresses the user's question.
- If multiple pages are relevant, combine information logically.
- Do NOT hallucinate facts—only use information from retrieved pages.

**Step 5 — Cite Sources**
At the end of your final answer, always list the Wikipedia pages you used:
- Page Title — URL

**Failure case**
If no relevant results are found:
- State that no useful Wikipedia information could be located based on the attempted search.

-----------------------------
ADDITIONAL RULES
-----------------------------
- Do not fabricate facts or URLs.
- Only rely on information retrieved through the provided tools.
- Be precise and helpful in your synthesis.
- Keep your response well-structured and easy to read.
""".strip()



In [8]:
class Reference(BaseModel):
    title: str
    filename: str

class Section(BaseModel):
    heading: str
    content: str
    references: list[Reference]


class SearchResultArticle(BaseModel):
    found_answer: bool
    title: str
    sections: list[Section]
    references: list[Reference]

In [31]:
def force_answer_after_5_searches(messages: list[ModelMessage]) -> list[ModelMessage]: 
    num_tool_calls = 0

    for m in messages:
        for p in m.parts:
            if p.part_kind == 'tool-call' and p.tool_name == 'fetch_wikipedia_page':
                num_tool_calls = num_tool_calls + 1

    if num_tool_calls >= 5:
        print('forcing output')
        last_message = messages[-1]
        finish_prompt = 'System message: The maximal number of searches has exceeded 5. Proceed to finishing the writeup'
        finish_prompt_part = UserPromptPart(finish_prompt)
        last_message.parts.append(finish_prompt_part)

    return messages


In [10]:
@dataclass
class AgentConfig:
    model: str = "openai:gpt-4o-mini"
    WIKI_API = "https://en.wikipedia.org/w/api.php"
    HEADERS = {"User-Agent": "YourApp/0.1 (you@example.com)"}

In [ ]:
# class NamedCallback:

#     def __init__(self, agent):
#         self.agent_name = agent.name

#     async def print_function_calls(self, ctx, event):
#         # Detect nested streams
#         if hasattr(event, "__aiter__"):
#             async for sub in event:
#                 await self.print_function_calls(ctx, sub)
#             return

#         if isinstance(event, FunctionToolCallEvent):
#             tool_name = event.part.tool_name
#             args = event.part.args
#             print(f"TOOL CALL ({self.agent_name}): {tool_name}({args})")

#     async def __call__(self, ctx, event):
#         return await self.print_function_calls(ctx, event)

In [14]:
from pydantic_ai.messages import FunctionToolCallEvent, FunctionToolResultEvent

class NamedCallback:

    def __init__(self, agent):
        self.agent_name = agent.name

    async def print_function_calls(self, ctx, event):
        # Detect nested streams
        if hasattr(event, "__aiter__"):
            async for sub in event:
                await self.print_function_calls(ctx, sub)
            return

        if isinstance(event, FunctionToolCallEvent):
            print("TOOL CALL →", event.part.tool_name, event.part.args_as_dict(), event.tool_call_id)
        elif isinstance(event, FunctionToolResultEvent):
            print("TOOL RES  ←", event.result.tool_name, event.tool_call_id, event.result.content)

    async def __call__(self, ctx, event):
        return await self.print_function_calls(ctx, event)

In [12]:
def create_agent(config: AgentConfig = None) -> Agent:
    if config is None:
        config = AgentConfig()


    agent = Agent(
        name="wikiagent",
        instructions=instructions,
        tools=[tools.get_search_queries, tools.fetch_wiki_page],
        model=config.model,
        output_type=SearchResultArticle,
        history_processors=[force_answer_after_5_searches]
    )

    return agent

In [20]:

import asyncio
from dotenv import load_dotenv
load_dotenv()


True

In [ ]:
agent= create_agent()
agent_callback= NamedCallback(agent)
async def run_agent(user_prompt: str):
    results = await agent.run(
        user_prompt=user_prompt,
        event_stream_handler=agent_callback
    )

    return results

await run_agent("what is xcvwwq")

TOOL CALL → get_search_queries {'subject': 'xcvwwq'} call_z3bC2vGQuZqsNW70irZD6K9z
TOOL RES  ← get_search_queries call_z3bC2vGQuZqsNW70irZD6K9z []


AgentRunResult(output=SearchResultArticle(found_answer=False, title='', sections=[], references=[]))

In [30]:
from typing import List, Optional, Literal
from pydantic import BaseModel, Field, HttpUrl, ConfigDict

# ---------- Citations & references ----------

class Citation(BaseModel):
    """A single Wikipedia source used in the answer."""
    title: str = Field(..., min_length=1, description="Wikipedia page title")
    url: HttpUrl = Field(..., description="Canonical Wikipedia URL")

class Reference(BaseModel):
    """
    Optional supporting reference (local artifact).
    Useful if you also save extracted files/snippets.
    """
    title: str = Field(..., min_length=1)
    filename: str = Field(..., min_length=1)


# ---------- Structured sections ----------

class Section(BaseModel):
    """A structured section of the final write-up."""
    heading: str = Field(..., min_length=1)
    content: str = Field(..., min_length=1)
    references: List[Reference] = Field(default_factory=list)


# ---------- Tool I/O mirrors (handy if you want to store traces) ----------

# class SearchHit(BaseModel):
#     """Mirror of your SearchQuery tool output."""
#     title: str
#     snippet: str = ""

# class WebPageContent(BaseModel):
#     """Mirror of your fetch_wikipedia_page tool output."""
#     wiki_url: HttpUrl
#     title: str
#     content: str

# class SearchWikipediaInput(BaseModel):
#     subject: str = Field(..., min_length=1, max_length=200)

# class FetchWikipediaPageInput(BaseModel):
#     title: str = Field(..., min_length=1, max_length=300)
#     plain_text: bool = Field(True, description="If False, HTML body is expected")


# ---------- Final agent output ----------

class SearchResultArticle(BaseModel):
    """
    The final, user-facing answer from the Wikipedia agent.
    """
    model_config = ConfigDict(str_min_length=1)

    found_answer: bool = Field(..., description="True if sufficient Wikipedia info was found")
    question: str = Field(..., description="The user's original question")
    answer: str = Field(..., description="Concise synthesized answer for the user")
    sections: List[Section] = Field(default_factory=list, description="Optional structured breakdown")
    citations: List[Citation] = Field(default_factory=list, description="Wikipedia pages used")
    notes: Optional[str] = Field(None, description="Optional caveats/limits/assumptions")
    # Optional light telemetry for debugging (keep it non-PII)
    meta_calls_used: int = Field(0, ge=0, description="Number of fetch_wikipedia_page calls used")
    meta_search_hits_considered: int = Field(0, ge=0)


In [33]:
def create_agent(config: AgentConfig | None = None) -> Agent:
    config = config or AgentConfig()

    agent = Agent(
        name="wikipedia-searcher",
        instructions=instructions,  # your improved prompt string
        tools=[tools.get_search_queries, tools.fetch_wiki_page],
        model=config.model,
        output_type=SearchResultArticle,
        history_processors=[force_answer_after_5_searches],
    )
    return agent


In [35]:
agent= create_agent()
agent_callback= NamedCallback(agent)
async def run_agent(user_prompt: str):
    results = await agent.run(
        user_prompt=user_prompt,
        event_stream_handler=agent_callback
    )

    return results

await run_agent("what does capybara eat")

TOOL CALL → get_search_queries {'subject': 'capybara diet'} call_WCOLFUrnuzbn5OntrjbRfuTb
TOOL RES  ← get_search_queries call_WCOLFUrnuzbn5OntrjbRfuTb [SearchQuery(title='Capybara', snippet='The capybara or greater capybara (Hydrochoerus hydrochaeris) is the largest living rodent, native to South America. It is a member of the genus Hydrochoerus'), SearchQuery(title='Lesser capybara', snippet='The lesser capybara (Hydrochoerus isthmius) is a large semi-aquatic rodent found in South America that has vast similarities, yet subtle differences, with'), SearchQuery(title='Hydrochoerus', snippet='(khoîros), meaning "pig". Capybaras are semiaquatic, found in and near lakes, rivers, swamps, and flooded savanna. Their diets are dominated by grasses'), SearchQuery(title='Guinea pig', snippet="similar laws against keeping a guinea pig by itself. The guinea pig's natural diet is grass; their molars are particularly suited for grinding plant matter"), SearchQuery(title='List of caviids', snippet='f

AgentRunResult(output=SearchResultArticle(found_answer=True, question='what does capybara eat', answer='Capybaras are herbivores, primarily grazing on grasses, aquatic plants, fruits, and tree bark. They are selective feeders, consuming various types of plants depending on the season. During the wet season, they primarily eat grass, while in the dry season, they often switch to readily available reeds. Capybaras also practice autocoprophagy, which means they eat their own feces to obtain beneficial bacteria that aid in digesting cellulose from the grass. This diet helps them extract maximum nutrients from their food, including protein and vitamins. They can live for 8 to 10 years, but typically, their lifespan is shorter in the wild due to predation.', sections=[], citations=[Citation(title='Capybara', url=HttpUrl('https://en.wikipedia.org/wiki/Capybara')), Citation(title='Hydrochoerus', url=HttpUrl('https://en.wikipedia.org/wiki/Hydrochoerus'))], notes=None, meta_calls_used=0, meta_se

In [7]:
from wikipagent import create_agent, NamedCallback

In [8]:
agent= create_agent()
agent_callback= NamedCallback(agent)
async def run_agent(user_prompt: str):
    results = await agent.run(
        user_prompt=user_prompt,
        event_stream_handler=agent_callback
    )

    return results

await run_agent("how long does capybara live")

TOOL CALL → get_search_queries {'subject': 'capybara lifespan'} call_R1fwCzGzMZTXrW6Df5RcXtTn
TOOL RES  ← get_search_queries call_R1fwCzGzMZTXrW6Df5RcXtTn [SearchQuery(title='Capybara', snippet='The capybara or greater capybara (Hydrochoerus hydrochaeris) is the largest living rodent, native to South America. It is a member of the genus Hydrochoerus'), SearchQuery(title='Lesser capybara', snippet='2024-10-21. "Capybara | Description, Behavior, & Facts | Britannica". www.britannica.com. 2024-09-28. Retrieved 2024-11-10. "Capybara Definition, Lifespan & Reproduction"'), SearchQuery(title='Rodent', snippet='musculature and innervation in the region. While the largest species, the capybara, can weigh as much as 66\xa0kg (146\xa0lb), most rodents weigh less than 100\xa0g'), SearchQuery(title='Mammal', snippet='in numbers of species are Rodentia: mice, rats, porcupines, beavers, capybaras, and other gnawing mammals; Chiroptera: bats; and Eulipotyphla: shrews'), SearchQuery(title='List of Wil

AgentRunResult(output=SearchResultArticle(found_answer=True, question='how long does capybara live', answer='The maximum lifespan of a capybara is typically around 8 to 10 years. However, in the wild, they usually do not live longer than 4 years due to predation. In captivity, capybaras can live up to 12 years.', sections=[], citations=[Citation(title='Capybara', url=HttpUrl('https://en.wikipedia.org/wiki/Capybara'))], notes=None))

In [9]:
agent= create_agent()
agent_callback= NamedCallback(agent)
async def run_agent(user_prompt: str):
    results = await agent.run(
        user_prompt=user_prompt,
        event_stream_handler=agent_callback
    )

    return results

result= await run_agent("how long does mamoths live")

TOOL CALL → get_search_queries {'subject': 'mammoth lifespan'} call_cT1rKPoowXSTPnEdSx1cf4v9
TOOL RES  ← get_search_queries call_cT1rKPoowXSTPnEdSx1cf4v9 [SearchQuery(title='Woolly mammoth', snippet='The woolly mammoth (Mammuthus primigenius) is an extinct species of mammoth that lived from the Middle Pleistocene until its extinction in the Holocene'), SearchQuery(title='Columbian mammoth', snippet='The Columbian mammoth (Mammuthus columbi) is an extinct species of mammoth that inhabited North America from southern Canada to Costa Rica during the Pleistocene'), SearchQuery(title='The Way Things Work (TV series)', snippet='science to young viewers and revolves around the residents of the backward Mammoth Island as they struggle through daily life with the use of outlandish contraptions'), SearchQuery(title='List of longest-living organisms', snippet="only the observed or estimated length of an individual organism's natural lifespan – that is, the duration of time between its birth or co

In [12]:
result

AgentRunResult(output=SearchResultArticle(found_answer=True, question='how long does mammoths live', answer='Mammoths, specifically woolly and Columbian mammoths, likely had lifespans similar to modern elephants. Woolly mammoths could probably live up to around 60 years, while Columbian mammoths may have had a lifespan of about 80 years.', sections=[], citations=[Citation(title='Woolly mammoth', url=HttpUrl('https://en.wikipedia.org/wiki/Woolly_mammoth')), Citation(title='Columbian mammoth', url=HttpUrl('https://en.wikipedia.org/wiki/Columbian_mammoth'))], notes=None))

In [13]:
output= result.output

In [15]:
print(output.format_markdown( ))

# how long does mammoths live

## Answer

Mammoths, specifically woolly and Columbian mammoths, likely had lifespans similar to modern elephants. Woolly mammoths could probably live up to around 60 years, while Columbian mammoths may have had a lifespan of about 80 years.

## Sources (Wikipedia)
- [Woolly mammoth](https://en.wikipedia.org/wiki/Woolly_mammoth)
- [Columbian mammoth](https://en.wikipedia.org/wiki/Columbian_mammoth)


In [17]:
import main

In [18]:
result = await main.run_agent('Where did mandela live for most of his life?')

TOOL CALL → get_search_queries {'subject': 'Nelson Mandela life biography residence'} call_hKpUwa5BWKnBBlyMy25HRjGU
TOOL RES  ← get_search_queries call_hKpUwa5BWKnBBlyMy25HRjGU [SearchQuery(title='Winnie Mandela', snippet='a South African politician, anti-apartheid activist, second wife of Nelson Mandela. During her political career, she served as a Member of Parliament from'), SearchQuery(title='Nelson Mandela', snippet='Nelson Rolihlahla Mandela (/mænˈdɛlə/ man-DEL-ə, Xhosa: [xolíɬaɬa mandɛ̂ːla]; born Rolihlahla Mandela; 18 July 1918\xa0– 5 December 2013) was a South African'), SearchQuery(title='Mandela Rhodes Scholarship', snippet='(Rwanda & University of KwaZulu-Natal, 2006)". The Mandela Rhodes Foundation is one of Nelson Mandela’s three official legacy organisations, founded in 2003'), SearchQuery(title='Marike de Klerk', snippet='stay in the Libertas home and Nelson Mandela would take up residence in another home known as the Presidency. Mandela later said that he was under pre

In [19]:
output= result.output
print(output.format_markdown( ))

# Where did Mandela live for most of his life?

## Answer

Nelson Mandela lived for most of his life in several key locations in South Africa. He was born in the village of Mvezo in Umtata. After his early childhood, he spent much of his youth in Qunu, which is near Umtata. After moving to Johannesburg in the 1940s, he lived in various townships, specifically Orlando, where he began his political career. He was incarcerated for 27 years, spending 18 of those years on Robben Island, before being moved to Pollsmoor Prison, and later to Victor Verster Prison. In the final years of his life, Mandela lived in Houghton, a suburb of Johannesburg, and frequently returned to his home village of Qunu before his death in 2013.

## Sources (Wikipedia)
- [Nelson Mandela](https://en.wikipedia.org/wiki/Nelson_Mandela)


In [33]:
from pydantic import BaseModel, Field, HttpUrl, ConfigDict

In [34]:
isinstance(output.citations[0].url, HttpUrl)

True

In [23]:
import json
from dataclasses import dataclass


@dataclass
class ToolCall:
    name: str
    args: dict


def get_tool_calls(result, tool_name) -> list[ToolCall]:
    calls = []

    for m in result.new_messages():
        for p in m.parts:
            kind = p.part_kind
            if kind == 'tool-call':
                if p.tool_name == 'final_result':
                    continue # skip final result tool calls
                if p.tool_name != tool_name:
                    continue
                call = ToolCall(
                    name=p.tool_name,
                    args=json.loads(p.args)
                )
                calls.append(call)

    return calls

In [24]:
get_tool_calls(result, 'fetch_wiki_page')

[ToolCall(name='fetch_wiki_page', args={'title': 'Nelson Mandela'})]